<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-11-02 17:53:50


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=bcb46fc3ec16a1c0661fea4c51e0b31b6f852e4128e24cd2e994fe8fbc5e2d43
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    68
1    62
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    28
V40     21
VR       5
AR       4
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    120
0     10
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    25
0    24
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%Min'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-10-04 00:00:00+05:30,19436.0,18507.0,5.02,^NSEI,20192.35,16945.05,14.70,40.0
2023-10-11 00:00:00+05:30,19811.0,18538.0,6.87,^NSEI,20192.35,16945.05,16.92,56.0
2023-10-18 00:00:00+05:30,19671.0,18580.0,5.87,^NSEI,20192.35,16945.05,16.09,49.0
2023-10-26 00:00:00+05:30,18857.0,18608.0,1.34,^NSEI,20192.35,16945.05,11.28,27.0
2023-11-02 00:00:00+05:30,19133.0,18635.0,2.67,^NSEI,20192.35,16945.05,12.91,41.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Min', 'Close', 'BizCheck',	'FinCheck',	'TechCheck'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# v40 and v40n
df_tmp = df_prospects[df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,19.50,46.18,60.0,71.5,31.40,23.40,V40N,NaN,1,NaN,1,M,NaN,5.0
ABBOTINDIA.NS,0.98,17.16,45.0,45.9,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,10.0
AKZOINDIA.NS,0.72,19.15,43.0,27.9,34.70,26.10,V40,BTT,1,NaN,1,H,NaN,19.0
ANGELONE.NS,76.52,169.32,81.0,21.7,44.00,47.10,V40N,NaN,1,NaN,1,M,FINANCE,-0.0
ASTRAZEN.NS,25.27,49.12,54.0,70.8,30.90,23.00,V40N,NaN,1,NaN,1,M,NaN,2.0
AXISBANK.NS,5.69,19.40,47.0,12.9,6.15,14.50,V40,NaN,1,NaN,1,H,BANKS,6.0
BAJAJ-AUTO.NS,22.71,59.87,59.0,22.0,26.20,20.20,V40,NaN,1,NaN,1,H,NaN,3.0
BAJAJHLDNG.NS,3.48,24.60,48.0,13.8,9.71,11.20,V40,NaN,1,NaN,0,H,NaN,9.0
BAJFINANCE.NS,9.69,35.75,38.0,34.4,11.80,23.50,V40,BTT,1,NaN,1,H,FINANCE,10.0


In [15]:
# all other prospects
df_tmp = df_prospects[~df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
ACC.NS,-0.74,16.69,36.0,26.70,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,46.0
ADANIENT.NS,-2.71,85.71,33.0,96.90,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,88.0
ADANIGREEN.NS,-8.60,93.01,41.0,96.90,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,233.0
ADANIPORTS.NS,7.54,68.56,39.0,22.60,9.53,14.30,NaN,NaN,1,NaN,1,L,NaN,25.0
ALKYLAMINE.NS,-12.29,0.44,31.0,57.80,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,114.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VGUARD.NS,9.00,23.04,45.0,59.60,15.20,12.60,NaN,NaN,1,NaN,0,L,NaN,7.0
VOLTAS.NS,1.48,13.16,45.0,90.40,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,58.0
WSTCSTPAPR.NS,31.22,59.54,53.0,4.75,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,5.0


In [16]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
BLUESTARCO.NS,22.96,64.59,61.0,60.50,24.40,23.40,AR,BTT,1,NaN,1,L,AC,63.0
UPL.NS,-18.32,1.64,28.0,23.80,14.30,13.40,AR,BTT,1,NaN,0,L,NaN,51.0
PNB.NS,30.26,91.61,53.0,13.90,4.10,3.34,VR,BTT,1,NaN,1,L,BANKS,28.0
AKZOINDIA.NS,0.72,19.15,43.0,27.90,34.70,26.10,V40,BTT,1,NaN,1,H,NaN,19.0
UJJIVANSFB.NS,39.61,132.98,47.0,1000.00,1000.00,1000.00,V40N,BTT,1,NaN,1,M,BANKS,13.0
MOTILALOFS.NS,32.86,78.09,54.0,10.00,12.90,15.60,V40N,BTT,1,NaN,1,M,FINANCE,11.0
ABBOTINDIA.NS,0.98,17.16,45.0,45.90,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,10.0
BAJFINANCE.NS,9.69,35.75,38.0,34.40,11.80,23.50,V40,BTT,1,NaN,1,H,FINANCE,10.0
J&KBANK.NS,55.44,173.83,51.0,7.31,4.92,13.10,VR,BTT,1,NaN,1,L,BANKS,9.0


In [17]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
VINATIORGA.NS,-7.95,0.00,22.0,39.8,30.40,22.6,V40N,NaN,1,NaN,1,M,NaN,37.0
AKZOINDIA.NS,0.72,19.15,43.0,27.9,34.70,26.1,V40,BTT,1,NaN,1,H,NaN,19.0
ITC.NS,2.65,34.88,39.0,26.7,39.00,29.1,V40,NaN,1,NaN,1,H,FMCG,14.0
ABBOTINDIA.NS,0.98,17.16,45.0,45.9,41.10,31.6,V40,BTT,1,NaN,1,H,PHARMA,10.0
MARICO.NS,1.37,14.11,38.0,49.1,41.90,35.8,V40,NaN,1,NaN,1,H,FMCG,10.0
ICICIBANK.NS,0.54,13.50,40.0,16.0,6.32,17.2,V40,NaN,1,NaN,1,H,BANKS,8.0


In [18]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
ADANIGREEN.NS,-8.60,93.01,41.0,96.9,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,233.0
BLUESTARCO.NS,22.96,64.59,61.0,60.5,24.40,23.40,AR,BTT,1,NaN,1,L,AC,63.0
IGL.NS,-13.94,2.28,32.0,15.1,26.60,20.50,NaN,NaN,1,NaN,1,L,NaN,44.0
KANSAINER.NS,5.75,25.62,35.0,41.5,14.50,10.70,V40N,NaN,1,NaN,1,M,NaN,40.0
VINATIORGA.NS,-7.95,0.00,22.0,39.8,30.40,22.60,V40N,NaN,1,NaN,1,M,NaN,37.0
PNB.NS,30.26,91.61,53.0,13.9,4.10,3.34,VR,BTT,1,NaN,1,L,BANKS,28.0
MUTHOOTFIN.NS,15.26,46.43,61.0,13.8,12.10,17.90,NaN,NaN,1,NaN,1,L,FINANCE,26.0
ADANIPORTS.NS,7.54,68.56,39.0,22.6,9.53,14.30,NaN,NaN,1,NaN,1,L,NaN,25.0
TASTYBITE.NS,31.36,103.57,50.0,1000.0,1000.00,1000.00,NaN,NaN,1,NaN,1,L,NaN,25.0


In [19]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
JCHAC.NS,-5.13,13.54,31.0,1000.0,1000.0,1000.0,V40N,NaN,1,NaN,0,M,NaN,201.0
SFL.NS,-2.26,15.68,49.0,56.7,13.6,12.5,V40N,NaN,1,NaN,0,M,NaN,79.0
SYMPHONY.NS,-10.25,2.52,40.0,50.8,14.8,13.5,V40N,NaN,1,NaN,0,M,NaN,74.0
LALPATHLAB.NS,15.59,39.10,54.0,68.1,18.2,15.0,V40N,NaN,1,NaN,0,M,NaN,69.0
BLUESTARCO.NS,22.96,64.59,61.0,60.5,24.4,23.4,AR,BTT,1,NaN,1,L,AC,63.0
DREAMFOLKS.NS,3.50,49.45,63.0,36.7,76.8,60.2,AR,ATH,0,NaN,0,L,MISC,54.0
PFIZER.NS,1.36,13.24,44.0,116.0,27.2,19.8,V40,NaN,1,NaN,0,H,PHARMA,54.0
UPL.NS,-18.32,1.64,28.0,23.8,14.3,13.4,AR,BTT,1,NaN,0,L,NaN,51.0
TTKPRESTIG.NS,4.72,20.79,49.0,49.0,18.1,13.8,V40N,NaN,1,NaN,0,M,NaN,47.0


In [20]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
YESBANK.NS,-5.59,8.81,33.0,55.2,4.94,1.99,NaN,NaN,1,NaN,0,L,NaN,1628.0
ADANIGREEN.NS,-8.60,93.01,41.0,96.9,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,233.0
JCHAC.NS,-5.13,13.54,31.0,1000.0,1000.00,1000.00,V40N,NaN,1,NaN,0,M,NaN,201.0
BIOCON.NS,-8.74,12.25,25.0,37.1,5.92,4.78,NaN,NaN,1,NaN,0,L,NaN,116.0
ALKYLAMINE.NS,-12.29,0.44,31.0,57.8,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,114.0
ADANIENT.NS,-2.71,85.71,33.0,96.9,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,88.0
SFL.NS,-2.26,15.68,49.0,56.7,13.60,12.50,V40N,NaN,1,NaN,0,M,NaN,79.0
SYMPHONY.NS,-10.25,2.52,40.0,50.8,14.80,13.50,V40N,NaN,1,NaN,0,M,NaN,74.0
DIVISLAB.NS,-0.54,22.53,21.0,60.1,19.40,14.90,NaN,NaN,1,NaN,0,L,PHARMA,58.0


In [21]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 8


,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
ANGELONE.NS,76.52,169.32,81.0,21.7,44.00,47.1,V40N,NaN,1,NaN,1,M,FINANCE,-0.0
COLPAL.NS,23.81,49.19,61.0,134.0,179.00,158.0,V40,BTT,1,NaN,1,H,FMCG,-0.0
ASTRAZEN.NS,25.27,49.12,54.0,70.8,30.90,23.0,V40N,NaN,1,NaN,1,M,NaN,2.0
EQUITASBNK.NS,23.80,101.61,59.0,14.6,8.04,12.2,V40N,NaN,1,NaN,1,M,BANKS,2.0
HCLTECH.NS,13.29,29.51,59.0,22.4,28.30,23.0,V40,NaN,1,NaN,1,H,IT,2.0
BAJAJ-AUTO.NS,22.71,59.87,59.0,22.0,26.20,20.2,V40,NaN,1,NaN,1,H,NaN,3.0
MCDOWELL-N.NS,18.79,43.72,58.0,62.4,20.00,17.5,V40N,NaN,1,NaN,1,M,BREWERIES,3.0
TITAN.NS,13.57,39.16,49.0,88.3,25.10,30.8,V40,NaN,1,NaN,1,H,JEWELLERY,4.0


In [22]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,19.50,46.18,60.0,71.50,31.40,23.40,V40N,NaN,1,NaN,1,M,NaN,5.0
ABBOTINDIA.NS,0.98,17.16,45.0,45.90,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,10.0
ACC.NS,-0.74,16.69,36.0,26.70,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,46.0
ADANIENT.NS,-2.71,85.71,33.0,96.90,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,88.0
ADANIGREEN.NS,-8.60,93.01,41.0,96.90,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,-6.31,7.60,30.0,62.00,27.80,29.00,V40N,NaN,1,NaN,0,M,NaN,25.0
VOLTAS.NS,1.48,13.16,45.0,90.40,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,58.0
WSTCSTPAPR.NS,31.22,59.54,53.0,4.75,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,5.0


In [23]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [24]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-10-04 00:00:00+05:30,560.0,525.0,6.71,BERGEPAINT.NS,710.11,447.11,25.30,38.0
2023-10-11 00:00:00+05:30,561.0,526.0,6.63,BERGEPAINT.NS,710.11,447.11,25.44,39.0
2023-10-18 00:00:00+05:30,572.0,528.0,8.40,BERGEPAINT.NS,710.11,447.11,28.01,46.0
2023-10-26 00:00:00+05:30,538.0,530.0,1.46,BERGEPAINT.NS,710.11,447.11,20.27,33.0
2023-11-02 00:00:00+05:30,549.0,532.0,3.19,BERGEPAINT.NS,710.11,447.11,22.78,41.0
